In [381]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [382]:
#####Pre-Processing for Bayes Model########
bayes16 = pd.read_csv('final16.csv', index_col=[0])
bayes18 = pd.read_csv('final18.csv', index_col=[0])


In [383]:
######Edit GDP Data#######
'''
gdpBayes16 = bayes16[['Country', 'GDP per Capita']]
gdpBayes16 = gdpBayes16.sort_values(by='GDP per Capita', ascending=True)

gdpData = go.Bar(x=gdpBayes16.Country, y=gdpBayes16['GDP per Capita'])
data = [gdpData]
layout = go.Layout(title='Countries ranked by GDP per Capita')
fig = go.Figure(data=data, layout=layout)
#fig.show()
'''

#######Edit the EPI Values#######
# Epi values are based on relative scale between 0-100
# Graph is mostly linear therefore its fine to bin them into categories based on quantile values
def discreteEPI(cat):
    cat = cat.apply(
    lambda x: 
    'Very Poor' if x <= float(20) else
    ('Poor' if x > float(20) and x <= float(40) else
    ('Average' if x > float(40) and x <= float(60) else
    ('Good' if x > float(60) and x <= float(80) else
    'Very Good'))))
    return cat

bayes16['Air Quality'] = discreteEPI(bayes16['Air Quality'])
bayes16['Biodiversity'] = discreteEPI(bayes16['Biodiversity'])
bayes16['Water Sanitation'] = discreteEPI(bayes16['Water Sanitation'])

#######Edit the GDP Values#######
# Convert GDP to discrete states using World Bank Classifications for specific year
# 2016 - https://blogs.worldbank.org/opendata/new-country-classifications-2016
# 2018 - https://blogs.worldbank.org/opendata/new-country-classifications-income-level-2018-2019
bayes16['GDP per Capita'] = bayes16['GDP per Capita'].apply(
    lambda x: 
    'Low-Income' if x <= float(1025) else
    ('Lower-Middle-Income' if x > float(1025) and x <= float(4035) else
    ('Upper-Middle-Income' if x > float(4035) and x <= float(12475) else
    'High-Income')))

#######Edit the CO2 Values#######
# CO2 values have a left skew there need to undergo a log transformation
bayes16['CO2 per Capita'] = (bayes16["CO2 per Capita"] + 1).apply(np.log)
bayes16['CO2 per Capita'] = pd.cut(bayes16['CO2 per Capita'], 3, labels=["Low", "Medium", "High"])

#######Edit the Happiness Scores#######
# Happiness values follow very linear trend therefore no need to normalize
bayes16['Happiness Score'] = pd.cut(
    bayes16['Happiness Score'], 5, 
    labels=["Very Unhappy", "Unhappy", "Average", "Happy", "Very Happy"])

#######Remove Coutnry Labels for training data set########
bayes16 = bayes16[['Region', 'GDP per Capita', 'CO2 per Capita', 'Air Quality', 'Biodiversity', 'Water Sanitation', 'Happiness Score']]

# Upload to csv format
bayes16.to_csv('trainingData.csv', index=False)

bayes16



,Region,GDP per Capita,CO2 per Capita,Air Quality,Biodiversity,Water Sanitation,Happiness Score
0,Western Europe,High-Income,Medium,Very Good,Very Good,Very Good,Very Happy
1,Western Europe,High-Income,Medium,Good,Very Good,Very Good,Very Happy
2,Western Europe,High-Income,High,Very Good,Good,Very Good,Very Happy
3,Western Europe,High-Income,Medium,Very Good,Very Good,Very Good,Very Happy
4,Western Europe,High-Income,Medium,Very Good,Very Good,Very Good,Very Happy
...,...,...,...,...,...,...,...
141,Sub-Saharan Africa,Low-Income,Low,Average,Good,Average,Very Unhappy
142,Sub-Saharan Africa,Low-Income,Low,Good,Good,Average,Very Unhappy
143,Southern Asia,Low-Income,Low,Good,Poor,Poor,Very Unhappy
144,Sub-Saharan Africa,Low-Income,Low,Good,Good,Poor,Very Unhappy


In [384]:
########Edit EPI data########
airBayes16 = bayes16[['Country', 'Air Quality']]
airBayes16 = airBayes16.sort_values(by='Air Quality', ascending=True)

airData = go.Bar(x=airBayes16.Country, y=airBayes16['Air Quality'])
data = [airData]
layout = go.Layout(title='Countries ranked by Air Quality')
fig = go.Figure(data=data, layout=layout)
#fig.show()

KeyError: "['Country'] not in index"

In [385]:
bioBayes16 = bayes16[['Country', 'Biodiversity']]
bioBayes16 = bioBayes16.sort_values(by='Biodiversity', ascending=True)

bioData = go.Bar(x=bioBayes16.Country, y=bioBayes16['Biodiversity'])
data = [bioData]
layout = go.Layout(title='Countries ranked by Biodiversity')
fig = go.Figure(data=data, layout=layout)
#fig.show()

KeyError: "['Country'] not in index"

In [386]:
co2Bayes16 = bayes16[['Country', 'CO2 per Capita']]
co2Bayes16 = co2Bayes16.sort_values(by='CO2 per Capita', ascending=True)

co2Data = go.Bar(x=co2Bayes16.Country, y=co2Bayes16['CO2 per Capita'])
data = [co2Data]
layout = go.Layout(title='Countries ranked by CO2 per Capita')
fig = go.Figure(data=data, layout=layout)
#fig.show()

KeyError: "['Country'] not in index"

In [387]:
waterBayes16 = bayes16[['Country', 'Water Sanitation']]
waterBayes16 = waterBayes16.sort_values(by='Water Sanitation', ascending=True)

waterData = go.Bar(x=waterBayes16.Country, y=waterBayes16['Water Sanitation'])
data = [waterData]
layout = go.Layout(title='Countries ranked by Water Sanitation')
fig = go.Figure(data=data, layout=layout)
#fig.show()

KeyError: "['Country'] not in index"

In [388]:
happyBayes16 = bayes16[['Country', 'Happiness Score']]
happyBayes16 = happyBayes16.sort_values(by='Happiness Score', ascending=True)

happyData = go.Bar(x=happyBayes16.Country, y=happyBayes16['Happiness Score'])
data = [happyData]
layout = go.Layout(title='Countries ranked by Happiness Score')
fig = go.Figure(data=data, layout=layout)
#fig.show()

KeyError: "['Country'] not in index"